In [1]:
import numpy as np
import pandas as pd

from scipy.stats import beta
from scipy import integrate

def moving_average(x, n=2):
    '''
      Moving average

      References
      ----------
      [1] https://stackoverflow.com/a/14314054/2097158

      Attributes
      ----------
      x: numpy array
      n: window size
    '''

    ret = np.cumsum(x, dtype=float)

    ret[n:] = ret[n:] - ret[:-n]

    return ret[n - 1:] / n

In [2]:
# Two classes: 0 and 1

# Number of rows for each class
N_0 = 1000
N_1 = 1000

# Random values generated from beta distributions
# Beta distributions were chosen, because they best represent
# distributions of values in the interval [0,1]
y_0 = [beta.rvs(1.5, 4.5, random_state=k) for k in range(0, N_0)]
y_1 = [beta.rvs(4, 2, random_state=k) for k in range(0, N_1)]

# Suffix to include in filenames
file_name_suffix = 'balanced'

# Linearly spaced values in the interval [0,1]
x = np.linspace(start=0., stop=1., num=50)

# Histograms
hist_0, bin_edges = np.histogram(y_0, bins=x)
hist_1, bin_edges = np.histogram(y_1, bins=x)

# bin centers (prob) are moving averages of bin edges
df = pd.DataFrame({
        'prob': moving_average(bin_edges),
        '0': hist_0,
        'TN': np.cumsum(hist_0),
        'FP': np.sum(hist_0) - np.cumsum(hist_0),
        '1': hist_1,
        'TP': np.sum(hist_1) - np.cumsum(hist_1),
        'FN': np.cumsum(hist_1),
     })

# True Positive Rate
df['TPR'] = df['TP'] / (df['TP'] + df['FN'])

# False Positive Rate
df['FPR'] = df['FP'] / (df['FP'] + df['TN'])

# Trapezoidal rule for integration 
# (sum up all rows to obtain final AUC value)
df['AUC'] = - df['TPR'].rolling(window=2).mean() * df['FPR'].diff(periods=1)

In [3]:
df.head()

,prob,0,TN,FP,1,TP,FN,TPR,FPR,AUC
0,0.010204,23,23,977,0,1000,0,1.0,0.977,NaN
1,0.030612,44,67,933,0,1000,0,1.0,0.933,0.044
2,0.051020,38,105,895,0,1000,0,1.0,0.895,0.038
3,0.071429,47,152,848,0,1000,0,1.0,0.848,0.047
4,0.091837,51,203,797,0,1000,0,1.0,0.797,0.051


In [4]:
df.tail()

,prob,0,TN,FP,1,TP,FN,TPR,FPR,AUC
44,0.908163,0,1000,0,21,50,950,0.050,0.0,-0.0
45,0.928571,0,1000,0,21,29,971,0.029,0.0,-0.0
46,0.948980,0,1000,0,15,14,986,0.014,0.0,-0.0
47,0.969388,0,1000,0,8,6,994,0.006,0.0,-0.0
48,0.989796,0,1000,0,6,0,1000,0.000,0.0,-0.0


In [5]:
df['AUC'].sum()

0.9218035

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1,
    cols=1
    )

data_dict = {}

idx = 30

data_dict['TN'] = {
  'x': df['prob'].iloc[:idx+1],
  'y': df['0'].iloc[:idx+1],
  'name': 'TN: True Negatives',
  'line': {'shape': 'hv', 'color': 'firebrick'},
  'mode': 'lines',
  'fill': 'tozeroy'
}

fig.add_trace(data_dict['TN'], row=1, col=1)

data_dict['FP'] = {
  'x': df['prob'].iloc[idx:],
  'y': df['0'].iloc[idx:],
  'name': 'FP: False Positives',
  'line': {'shape': 'hv', 'color': 'green'},
  'mode': 'lines',
  'fill': 'tozeroy'
}

fig.add_trace(data_dict['FP'], row=1, col=1)

data_dict['FN'] = {
  'x': df['prob'].iloc[:idx+1],
  'y': df['1'].iloc[:idx+1],
  'name': 'FN: False Negatives',
  'line': {'shape': 'hv', 'color': 'darkorange'},
  'mode': 'lines',
  'fill': 'tozeroy'
}

fig.add_trace(data_dict['FN'], row=1, col=1)

data_dict['TP'] = {
  'x': df['prob'].iloc[idx:],
  'y': df['1'].iloc[idx:],
  'name': 'TP: True Positives',
  'line': {'shape': 'hv', 'color': 'cornflowerblue'},
  'mode': 'lines',
  'fill': 'tozeroy'
}

fig.add_trace(data_dict['TP'], row=1, col=1)

y_max = max(np.max(df['0']), np.max(df['1'])) * 1.05

data_dict['threshold'] = {
  'x': [df['prob'].iloc[idx], df['prob'].iloc[idx]],
  'y': [0., y_max],
  'name': 'threshold',
  'line': {'color': 'grey', 'width': 3, 'dash': 'dot'},
  'mode': 'lines'
}

fig.add_trace(data_dict['threshold'], row=1, col=1)

epsilon = 1.e-2

fig.update_xaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='Output Probability',
    row=1, col=1
)

fig.update_yaxes(
    range=[0, y_max],
    title='Frequency',
    row=1, col=1
)

fig.update_layout(
    font=dict(
        family='Courier New, monospace',
        size=20,
        color='Gray'
    )
)

fig.update_layout(
    autosize=False,
    width=900,
    height=600
)

fig.show()

fig.write_html('distr_{}.html'.format(file_name_suffix))

In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1,
    cols=2,
    horizontal_spacing=0.15,
    subplot_titles=('', '')
    )

data_dict = {}

data_dict['0'] = {
  'x': df['prob'],
  'y': df['0'],
  'name': 'class 0',
  'line': {'shape': 'hv', 'color': 'darkviolet'},
  'mode': 'lines',
  'fill': 'tozeroy'
}

fig.add_trace(data_dict['0'], row=1, col=1)

data_dict['1'] = {
  'x': df['prob'],
  'y': df['1'],
  'name': 'class 1',
  'line': {'shape': 'hv', 'color': 'cornflowerblue'},
  'mode': 'lines',
  'fill': 'tozeroy'
}

fig.add_trace(data_dict['1'], row=1, col=1)

y_max = max(np.max(df['0']), np.max(df['1'])) * 1.05

data_dict['threshold'] = {
  'x': [0, 0],
  'y': [0., y_max],
  'name': 'threshold',
  'line': {'color': 'orange', 'width': 4},
  'mode': 'lines'
}

fig.add_trace(data_dict['threshold'], row=1, col=1)

epsilon = 1.e-2

fig.update_xaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='Output Probability',
    row=1, col=1
)

fig.update_yaxes(
    range=[0, y_max],
    title='Frequency',
    row=1, col=1
)

data_dict['TPR'] = {
  'x': df['prob'],
  'y': df['TPR'],
  'name': 'TPR',
  'text': df['prob'],
  'line': {'shape': 'hv', 'color': 'firebrick', 'width': 3},
  'mode': 'lines'
}

fig.add_trace(data_dict['TPR'], row=1, col=2)

data_dict['FPR'] = {
  'x': df['prob'],
  'y': df['FPR'],
  'name': 'FPR',
  'text': df['prob'],
  'line': {'shape': 'hv', 'color': 'olive', 'width': 3},
  'mode': 'lines'
}

fig.add_trace(data_dict['FPR'], row=1, col=2)

epsilon = 5.e-2

fig.update_xaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='Threshold',
    row=1, col=2
)

fig.update_yaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='',
    row=1, col=2
)

data_dict['TPR0'] = {
  'x': [df['prob'].iloc[0]],
  'y': [df['TPR'].iloc[0]],
  'name': 'TPR at threshold',
  'marker': {'color': 'orange', 'size': 20, 'opacity': 0.6},
  'mode': 'markers'
}

fig.add_trace(data_dict['TPR0'], row=1, col=2)

data_dict['FPR0'] = {
  'x': [df['prob'].iloc[0]],
  'y': [df['FPR'].iloc[0]],
  'name': 'FPR at threshold',
  'marker': {'color': 'orangered', 'size': 20, 'opacity': 0.6},
  'mode': 'markers'
}

fig.add_trace(data_dict['FPR0'], row=1, col=2)

fig.update_layout(
    font=dict(
        family='Courier New, monospace',
        size=20,
        color='Gray'
    )
)

fig.update_layout(
    autosize=False,
    width=1200,
    height=600
)

n_frames = len(df['FPR'])

frames = [
  {'name': k,
   'data': [
              data_dict['0'],
              data_dict['1'],
              {
                'x': [df['prob'].iloc[k], df['prob'].iloc[k]],
                'y': [0., y_max],
                'name': 'threshold',
                'line': {'color': 'orange', 'width': 4},
                'mode': 'lines'
              },
              data_dict['TPR'],
              data_dict['FPR'],
              {
                'x': [df['prob'].iloc[k]],
                'y': [df['TPR'].iloc[k]],
                'name': 'TPR at threshold',
                'marker': {'color': 'orange', 'size': 20, 'opacity': 0.6},
                'mode': 'markers'
              },
              {
                'x': [df['prob'].iloc[k]],
                'y': [df['FPR'].iloc[k]],
                'name': 'FPR at threshold',
                'marker': {'color': 'orangered', 'size': 20, 'opacity': 0.6},
                'mode': 'markers'
              }
   ],
   'traces': list(range(0, 7))
   }
   for k in range(0, n_frames)
]

updatemenus = [{
    'buttons': [
      {
        'args': [[f'{k}' for k in range(n_frames)],
          {
              'easing': 'linear',
              'frame': {'duration': 500, 'redraw': False},
              'fromcurrent': True,
              'mode': 'immediate',
              'transition': {'duration': 0}
           }
          ],
        'label': 'Play',
        'method': 'animate'
      },
      {
          'args': [[None],
            {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
             }
            ],
          'label': 'Pause',
          'method': 'animate'
      }        
        ],
    'direction': 'left',
    'pad': {'r': 49, 't': 85},
    'showactive': True,
    'type': 'buttons',
    'x': 0.1,
    'xanchor': 'right',
    'y': -0.1,
    'yanchor': 'top'
}]

sliders = [{'yanchor': 'top',
            'xanchor': 'left', 
            'currentvalue': {
                'font': {'size': 16},
                'prefix': 'Threshold: ',
                'visible': True,
                'xanchor': 'right'
                },
            'transition': {'duration': 500.0, 'easing': 'linear'},
            'pad': {'b': n_frames, 't': 50}, 
            'len': 0.9, 'x': 0.1, 'y': -0.1, 
            'steps': [{
                'args': [[k], {
                    'frame': {
                        'duration': 500.0,
                        'easing': 'linear',
                        'redraw': False
                        },
                    'transition': {'duration': 0, 'easing': 'linear'}
                    }], 
                'label': '{:.2f}'.format(df['prob'].iloc[k]),
                'method': 'animate'
                } for k in range(n_frames)       
              ]
            }]

fig.update(frames=frames)

fig.update_layout(
    updatemenus=updatemenus,
    sliders=sliders
)

fig.show()

fig.write_html('TPR_FPR_{}.html'.format(file_name_suffix))

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1,
    cols=2,
    horizontal_spacing=0.15,
    subplot_titles=('', 'Find max diff to determine optimal threshold')
    )

data_dict = {}

data_dict['0'] = {
  'x': df['prob'],
  'y': df['0'],
  'name': 'class 0',
  'line': {'shape': 'hv', 'color': 'darkviolet'},
  'mode': 'lines',
  'fill': 'tozeroy'
}

fig.add_trace(data_dict['0'], row=1, col=1)

data_dict['1'] = {
  'x': df['prob'],
  'y': df['1'],
  'name': 'class 1',
  'line': {'shape': 'hv', 'color': 'cornflowerblue'},
  'mode': 'lines',
  'fill': 'tozeroy'
}

fig.add_trace(data_dict['1'], row=1, col=1)

y_max = max(np.max(df['0']), np.max(df['1'])) * 1.05

data_dict['threshold'] = {
  'x': [0, 0],
  'y': [0., y_max],
  'name': 'threshold',
  'line': {'color': 'orange', 'width': 4},
  'mode': 'lines'
}

fig.add_trace(data_dict['threshold'], row=1, col=1)

epsilon = 1.e-2

fig.update_xaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='Output Probability',
    row=1, col=1
)

fig.update_yaxes(
    range=[0, y_max],
    title='Frequency',
    row=1, col=1
)

data_dict['TPR'] = {
  'x': df['prob'],
  'y': df['TPR'],
  'name': 'TPR',
  'text': df['prob'],
  'line': {'shape': 'hv', 'color': 'firebrick', 'width': 3},
  'mode': 'lines'
}

fig.add_trace(data_dict['TPR'], row=1, col=2)

data_dict['FPR'] = {
  'x': df['prob'],
  'y': df['FPR'],
  'name': 'FPR',
  'text': df['prob'],
  'line': {'shape': 'hv', 'color': 'olive', 'width': 3},
  'mode': 'lines'
}

fig.add_trace(data_dict['FPR'], row=1, col=2)

epsilon = 5.e-2

fig.update_xaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='Threshold',
    row=1, col=2
)

fig.update_yaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='',
    row=1, col=2
)

data_dict['TPR0'] = {
  'x': [df['prob'].iloc[0]],
  'y': [df['TPR'].iloc[0]],
  'name': 'TPR at threshold',
  'marker': {'color': 'orange', 'size': 20, 'opacity': 0.6},
  'mode': 'markers'
}

fig.add_trace(data_dict['TPR0'], row=1, col=2)

data_dict['FPR0'] = {
  'x': [df['prob'].iloc[0]],
  'y': [df['FPR'].iloc[0]],
  'name': 'FPR at threshold',
  'marker': {'color': 'orangered', 'size': 20, 'opacity': 0.6},
  'mode': 'markers'
}

fig.add_trace(data_dict['FPR0'], row=1, col=2)

data_dict['diff0'] = {
  'x': [df['prob'].iloc[0], df['prob'].iloc[0]],
  'y': [df['FPR'].iloc[0], df['TPR'].iloc[0]],
  'name': 'Diff at threshold',
  'line': {'color': 'orange', 'width': 4},
  'mode': 'lines'
}

fig.add_trace(data_dict['diff0'], row=1, col=2)

fig.update_layout(
    font=dict(
        family='Courier New, monospace',
        size=20,
        color='Gray'
    )
)

fig.update_layout(
    autosize=False,
    width=1200,
    height=600
)

n_frames = len(df['FPR'])

frames = [
  {'name': k,
   'data': [
              data_dict['0'],
              data_dict['1'],
              {
                'x': [df['prob'].iloc[k], df['prob'].iloc[k]],
                'y': [0., y_max],
                'name': 'threshold',
                'line': {'color': 'orange', 'width': 4},
                'mode': 'lines'
              },
              data_dict['TPR'],
              data_dict['FPR'],
              {
                'x': [df['prob'].iloc[k]],
                'y': [df['TPR'].iloc[k]],
                'name': 'TPR at threshold',
                'marker': {'color': 'orange', 'size': 20, 'opacity': 0.6},
                'mode': 'markers'
              },
              {
                'x': [df['prob'].iloc[k]],
                'y': [df['FPR'].iloc[k]],
                'name': 'FPR at threshold',
                'marker': {'color': 'orangered', 'size': 20, 'opacity': 0.6},
                'mode': 'markers'
              },
              {
                'x': [df['prob'].iloc[k], df['prob'].iloc[k]],
                'y': [df['FPR'].iloc[k], df['TPR'].iloc[k]],
                'name': 'Diff at threshold',
                'text': (df['FPR'].iloc[k] - df['TPR'].iloc[k]),
                'line': {'color': 'orange', 'width': 4},
                'mode': 'lines'
              }
   ],
   'traces': list(range(0, 8))
   }
   for k in range(0, n_frames)
]

updatemenus = [{
    'buttons': [
      {
        'args': [[f'{k}' for k in range(n_frames)],
          {
              'easing': 'linear',
              'frame': {'duration': 500, 'redraw': False},
              'fromcurrent': True,
              'mode': 'immediate',
              'transition': {'duration': 0}
           }
          ],
        'label': 'Play',
        'method': 'animate'
      },
      {
          'args': [[None],
            {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
             }
            ],
          'label': 'Pause',
          'method': 'animate'
      }        
        ],
    'direction': 'left',
    'pad': {'r': 49, 't': 85},
    'showactive': True,
    'type': 'buttons',
    'x': 0.1,
    'xanchor': 'right',
    'y': -0.1,
    'yanchor': 'top'
}]

sliders = [{'yanchor': 'top',
            'xanchor': 'left', 
            'currentvalue': {
                'font': {'size': 16},
                'prefix': 'Threshold: ',
                'visible': True,
                'xanchor': 'right'
                },
            'transition': {'duration': 500.0, 'easing': 'linear'},
            'pad': {'b': n_frames, 't': 50}, 
            'len': 0.9, 'x': 0.1, 'y': -0.1, 
            'steps': [{
                'args': [[k], {
                    'frame': {
                        'duration': 500.0,
                        'easing': 'linear',
                        'redraw': False
                        },
                    'transition': {'duration': 0, 'easing': 'linear'}
                    }], 
                'label': '{:.2f}'.format(df['prob'].iloc[k]),
                'method': 'animate'
                } for k in range(n_frames)       
              ]
            }]

fig.update(frames=frames)

fig.update_layout(
    updatemenus=updatemenus,
    sliders=sliders
)

fig.show()

fig.write_html('TPR_FPR_diff_{}.html'.format(file_name_suffix))

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1,
    cols=2,
    horizontal_spacing=0.15,
    subplot_titles=('', 'AUC = {:.3f}'.format(df['AUC'].sum()))
    )

data_dict = {}

data_dict['0'] = {
  'x': df['prob'],
  'y': df['0'],
  'name': 'class 0',
  'line': {'shape': 'hv', 'color': 'darkviolet'},
  'mode': 'lines',
  'fill': 'tozeroy'
}

fig.add_trace(data_dict['0'], row=1, col=1)

data_dict['1'] = {
  'x': df['prob'],
  'y': df['1'],
  'name': 'class 1',
  'line': {'shape': 'hv', 'color': 'cornflowerblue'},
  'mode': 'lines',
  'fill': 'tozeroy'
}

fig.add_trace(data_dict['1'], row=1, col=1)

y_max = max(np.max(df['0']), np.max(df['1'])) * 1.05

data_dict['threshold'] = {
  'x': [0, 0],
  'y': [0., y_max],
  'name': 'threshold',
  'line': {'color': 'orange', 'width': 4},
  'mode': 'lines'
}

fig.add_trace(data_dict['threshold'], row=1, col=1)

epsilon = 1.e-2

fig.update_xaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='Output Probability',
    row=1, col=1
)

fig.update_yaxes(
    range=[0, y_max],
    title='Frequency',
    row=1, col=1
)

data_dict['ROC'] = {
  'x': df['FPR'],
  'y': df['TPR'],
  'name': 'ROC',
  'text': df['prob'],
  'line': {'shape': 'hv', 'color': 'firebrick', 'width': 3},
  'mode': 'lines'
}

fig.add_trace(data_dict['ROC'], row=1, col=2)

data_dict['baseline'] = {
  'x': np.linspace(0., 1., 100),
  'y': np.linspace(0., 1., 100),
  'name': 'Baseline',
  'line': {'color': 'gray', 'dash': 'dash'},
  'mode': 'lines'
}

fig.add_trace(data_dict['baseline'], row=1, col=2)

epsilon = 5.e-2

fig.update_xaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='FPR',
    row=1, col=2
)

fig.update_yaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='TPR',
    row=1, col=2
)

data_dict['t0'] = {
  'x': [df['FPR'].iloc[0]],
  'y': [df['TPR'].iloc[0]],
  'name': 'ROC at threshold',
  'marker': {'color': 'orange', 'size': 20, 'opacity': 0.6},
  'mode': 'markers'
}

fig.add_trace(data_dict['t0'], row=1, col=2)

fig.update_layout(
    font=dict(
        family='Courier New, monospace',
        size=20,
        color='Gray'
    )
)

fig.update_layout(
    autosize=False,
    width=1200,
    height=600
)

n_frames = len(df['FPR'])

frames = [
  {'name': k,
   'data': [
              data_dict['0'],
              data_dict['1'],
              {
                'x': [df['prob'].iloc[k], df['prob'].iloc[k]],
                'y': [0., y_max],
                'name': 'threshold',
                'line': {'color': 'orange', 'width': 4},
                'mode': 'lines'
              },
              data_dict['ROC'],
              data_dict['baseline'],
              {
                'x': [df['FPR'].iloc[k]],
                'y': [df['TPR'].iloc[k]],
                'name': 'ROC at threshold',
                'marker': {'color': 'orange', 'size': 20, 'opacity': 0.6},
                'mode': 'markers'
              }
   ],
   'traces': list(range(0, 6))
   }
   for k in range(0, n_frames)
]

updatemenus = [{
    'buttons': [
      {
        'args': [[f'{k}' for k in range(n_frames)],
          {
              'easing': 'linear',
              'frame': {'duration': 500, 'redraw': False},
              'fromcurrent': True,
              'mode': 'immediate',
              'transition': {'duration': 0}
           }
          ],
        'label': 'Play',
        'method': 'animate'
      },
      {
          'args': [[None],
            {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
             }
            ],
          'label': 'Pause',
          'method': 'animate'
      }        
        ],
    'direction': 'left',
    'pad': {'r': 49, 't': 85},
    'showactive': True,
    'type': 'buttons',
    'x': 0.1,
    'xanchor': 'right',
    'y': -0.1,
    'yanchor': 'top'
}]

sliders = [{'yanchor': 'top',
            'xanchor': 'left', 
            'currentvalue': {
                'font': {'size': 16},
                'prefix': 'Threshold: ',
                'visible': True,
                'xanchor': 'right'
                },
            'transition': {'duration': 500.0, 'easing': 'linear'},
            'pad': {'b': n_frames, 't': 50}, 
            'len': 0.9, 'x': 0.1, 'y': -0.1, 
            'steps': [{
                'args': [[k], {
                    'frame': {
                        'duration': 500.0,
                        'easing': 'linear',
                        'redraw': False
                        },
                    'transition': {'duration': 0, 'easing': 'linear'}
                    }], 
                'label': '{:.2f}'.format(df['prob'].iloc[k]),
                'method': 'animate'
                } for k in range(n_frames)       
              ]
            }]

fig.update(frames=frames)

fig.update_layout(
    updatemenus=updatemenus,
    sliders=sliders
)

fig.show()

fig.write_html('ROC_{}.html'.format(file_name_suffix))

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1,
    cols=2,
    horizontal_spacing=0.15,
    subplot_titles=('Regular ROC', 'Rotated ROC')
    )

data_dict = {}

data_dict['ROC'] = {
  'x': df['FPR'],
  'y': df['TPR'],
  'name': 'ROC',
  'text': df['prob'],
  'line': {'shape': 'hv', 'color': 'firebrick', 'width': 3},
  'mode': 'lines'
}

fig.add_trace(data_dict['ROC'], row=1, col=1)

data_dict['baseline'] = {
  'x': np.linspace(0., 1., 100),
  'y': np.linspace(0., 1., 100),
  'name': 'Baseline',
  'line': {'color': 'gray', 'dash': 'dash'},
  'mode': 'lines'
}

fig.add_trace(data_dict['baseline'], row=1, col=1)

epsilon = 5.e-2

fig.update_xaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='FPR',
    row=1, col=1
)

fig.update_yaxes(
    range=[0 - epsilon, 1 + epsilon],
    title='TPR',
    row=1, col=1
)

data_dict['ROC_rotation'] = {
  'x': (df['TPR']+df['FPR'])/np.sqrt(2),
  'y': (df['TPR']-df['FPR'])/np.sqrt(2),
  'name': 'Rotated ROC',
  'text': df['prob'],
  'line': {
      'color': 'firebrick',
      'width': 3,
      'shape': 'spline'
    },
  'mode': 'lines'
}

fig.add_trace(data_dict['ROC_rotation'], row=1, col=2)

data_dict['baseline_rotation'] = {
  'x': np.linspace(0., (2./np.sqrt(2.)), 10),
  'y': np.linspace(0., 0., 10),
  'name': 'Baseline',
  'line': {'color': 'gray', 'dash': 'dash'},
  'mode': 'lines'
}

fig.add_trace(data_dict['baseline_rotation'], row=1, col=2)

epsilon = 5.e-2

fig.update_xaxes(
    range=[0 - epsilon, (2./np.sqrt(2.)) + epsilon],
    title='$\\displaystyle\\frac{\\text{TPR}+\\text{FPR}}{\\sqrt{2}}$',
    row=1, col=2
)

fig.update_yaxes(
    range=[0 - epsilon, (1./np.sqrt(2.)) + epsilon],
    title='$\\displaystyle\\frac{\\text{TPR}-\\text{FPR}}{\\sqrt{2}}$',
    row=1, col=2
)

fig.update_layout(
    font=dict(
        family='Courier New, monospace',
        size=20,
        color='Gray'
    )
)

fig.update_layout(
    autosize=False,
    width=1200,
    height=600
)

fig.update_layout(showlegend=False) 

fig.show()

fig.write_html('ROC_rotation_{}.html'.format(file_name_suffix))